# Read modified station XML files with extra metadata in Json format 

## Fei Zhang

2020-07-14

In [1]:
from obspy import read_inventory

our_new_station_xml ="OA.CF28_station_inv_modified_json.xml"  # add this stationxml into asdf file
# VS extracted from ASDF
#our_new_station_xml ="OA.CF28_station_inv_modified_json_extracted_from_ASDF.xml"

our_inv = read_inventory(our_new_station_xml,format='STATIONXML')


stn_meta = our_inv.networks[0].stations


In [2]:
print(type(stn_meta[0].extra))
print(stn_meta[0].extra)

<class 'obspy.core.util.attribdict.AttribDict'>
AttribDict({'metadata_ga': AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis/xmlns/1.0', 'value': '\n{\n    "network_code":"OA",\n    "station_code":"CF28",\n\n    "orient_correction": {\n        "start_dt": "2017-11-07T09:07:34.930000Z",\n        "end_dt":   "2018-08-23T03:52:29.528000Z",\n        "azimuth_correction": -5.0\n    },\n\n  "gps_clock_corrections": [\n    {\n      "date": "2018-01-04",\n      "seconds": -1.3375814425470127\n    },\n    {\n      "date": "2018-01-05",\n      "seconds": -1.110449564656099\n    },\n    {\n      "date": "2018-01-06",\n      "seconds": -0.9032476255118933\n    }\n  ]\n\n}\n'})})


In [3]:
print(len(stn_meta))
print(stn_meta[1].start_date)
print(stn_meta[1].end_date)

print(stn_meta[1].equipments)
print(stn_meta[1].extra)

3
2018-06-09T06:21:41.000000Z
2018-08-12T05:41:01.000000Z
[]
AttribDict({'metadata_ga': AttribDict({'namespace': 'https://github.com/GeoscienceAustralia/hiperseis/xmlns/1.0', 'value': '\n{\n    "network_code":"OA",\n    "station_code":"CF28",\n\n    "orient_correction": {\n        "start_dt": "2017-11-07T09:07:34.930000Z",\n        "end_dt":   "2018-08-23T03:52:29.528000Z",\n        "azimuth_correction": -5.0\n    },\n\n  "gps_clock_corrections": [\n    {\n      "date": "2018-08-12",\n      "seconds": -0.38272156676887814\n    },\n    {\n      "date": "2018-08-13",\n      "seconds": -0.38602834753320375\n    },\n    {\n      "date": "2018-08-14",\n      "seconds": -0.3893351282975293\n    }\n  ]\n\n}\n  '})})


In [19]:
# extra_meta = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value
extra_meta = stn_meta[2].extra.metadata_ga.value
print(type(extra_meta))

<class 'str'>


In [20]:
print(extra_meta)


{
    "network_code":"OA",
    "station_code":"CF28",

    "orient_correction": {
        "start_dt": "2017-11-07T09:07:34.930000Z",
        "end_dt":   "2018-08-23T03:52:29.528000Z",
        "azimuth_correction": -5.0
    },
  "gps_clock_corrections": [
    {
      "date": "2018-06-10",
      "seconds": -0.17439437861636772
    },
    {
      "date": "2018-06-11",
      "seconds": -0.17770115938069328
    },
    {
      "date": "2018-06-12",
      "seconds": -0.18100794014501886
    }
  ]

}
  


In [6]:
import json

mdata = json.loads(extra_meta)

print (mdata)
print(mdata.keys())
print(type(mdata['gps_clock_corrections']))

for corr in mdata['gps_clock_corrections']:
    print (corr["date"], corr["seconds"] )

{'network_code': 'OA', 'station_code': 'CF28', 'orient_correction': {'start_dt': '2017-11-07T09:07:34.930000Z', 'end_dt': '2018-08-23T03:52:29.528000Z', 'azimuth_correction': -5.0}, 'gps_clock_corrections': [{'date': '2018-01-04', 'seconds': -1.3375814425470127}, {'date': '2018-01-05', 'seconds': -1.110449564656099}, {'date': '2018-01-06', 'seconds': -0.9032476255118933}]}
dict_keys(['network_code', 'station_code', 'orient_correction', 'gps_clock_corrections'])
<class 'list'>
2018-01-04 -1.3375814425470127
2018-01-05 -1.110449564656099
2018-01-06 -0.9032476255118933


In [7]:

print (mdata['gps_clock_corrections'])

[{'date': '2018-01-04', 'seconds': -1.3375814425470127}, {'date': '2018-01-05', 'seconds': -1.110449564656099}, {'date': '2018-01-06', 'seconds': -0.9032476255118933}]


In [8]:
# https://pbpython.com/pandas-list-dict.html

import pandas as pd

df_clock_correction = pd.DataFrame(mdata['gps_clock_corrections'])
df_clock_correction.head()

,date,seconds
0,2018-01-04,-1.337581
1,2018-01-05,-1.110450
2,2018-01-06,-0.903248


In [9]:
print (df_clock_correction.tail())

         date   seconds
0  2018-01-04 -1.337581
1  2018-01-05 -1.110450
2  2018-01-06 -0.903248


In [10]:
# get correction for a certain date YYYY-MM-DD
print(df_clock_correction.loc[df_clock_correction["date"] =="2018-01-04"])

         date   seconds
0  2018-01-04 -1.337581


In [11]:
type(df_clock_correction.query('date == "2018-01-04" ').seconds)

pandas.core.series.Series

In [12]:
df_clock_correction["date"] =="2018-01-04"

0     True
1    False
2    False
Name: date, dtype: bool

In [13]:
df_clock_correction.query('date == "2018-01-04" ').seconds.values[0]

-1.3375814425470127

In [14]:
df_clock_correction.describe()


,seconds
count,3.000000
mean,-1.117093
std,0.217243
min,-1.337581
25%,-1.224016
50%,-1.110450
75%,-1.006849
max,-0.903248


In [15]:
df_clock_correction.dtypes

date        object
seconds    float64
dtype: object

In [16]:
# What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml, and OA.CF28 
stations =our_inv.networks[0].stations
print (len(stations))

3


# Check the output statoinXML file from ASDF

We have added the input station xml into an ASDF file, then extract it out. See pyasdf_tests.ipynb

## https://github.com/SeismicData/pyasdf/issues/63

In [21]:
from obspy import read_inventory

our_new_station_xml ="OA.CF28_station_inv_modified_json_extracted_from_ASDF.xml"
our_inv = read_inventory(our_new_station_xml,format='STATIONXML')
# print(our_inv.networks[0].stations[0].extra)

stn_meta0 = our_inv.networks[0].stations[0]

In [18]:

print(type(stn_meta0.extra))
print(stn_meta0.extra)

AttributeError: 'Station' object has no attribute 'extra'

In [ ]:
print(stn_meta0.start_date)
print(stn_meta0.end_date)

print(stn_meta0.equipments)

# Dealing with multiple station-nodes in a network.station inventory file

See Jira: https://gajira.atlassian.net/browse/PV-130

 What if there are more than one Station Node, such as AU.HTT in testdata/network_AU_0.xml. 
 And our_new_station_xml ="OA.CF28_station_inv_modified_json.xml"
 
The multple station code represent the same station with different channel configuraiton over different time periods.
 

In [ ]:

# create the modified xml file by running
# python  add_time_corrections.py  /g/data/ha3/Passive/SHARED_DATA/GPS_Clock/corrections/AU.HTT_clock_correction.csv ../../tests/testdata/network_AU_0.xml 

def inspect_stations( our_new_station_xml ):
    
    our_inv = read_inventory(our_new_station_xml,format='STATIONXML')

    #csv_str = our_inv.networks[0].stations[0].extra.gpsclockcorrection.value

    stations =our_inv.networks[0].stations
    
    if len(stations)>=2:
        print (our_new_station_xml)
        print ("Warning: more than one Station Node = %s" %len(stations))
    
    return stations


In [ ]:
#our_new_station_xml ="/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/AU.HTT_station_inv_modified.xml"
inspect_stations(our_new_station_xml)

In [ ]:

import os, glob

xmlfolder = "/g/data/ha3/Passive/SHARED_DATA/GPS_Clock/StationXML_with_time_corrections2/"

for axmlfile in glob.glob(os.path.join(xmlfolder,"*.xml")):
    
    stations = inspect_stations(axmlfile)
    
#     if len(stations)>=2:
#         for astation in stations:
#             print("### ", astation.code, astation.extra)